# Pytorch tutorials

Tutotial ngắn về pytorch

Author: imthanhlv@gmail.com

## 1. Giới thiệu

Trong notebook này các bạn sẽ được làm quen với `Pytorch` và xây dựng mô hình seq2seq.

`Pytorch` là một ML framework phổ biến trong giới nghiên cứu về ML, DL vì tính linh hoạt và ổn định của API, độ chi tiết của document & ecosystem xung quanh pytorch. 

Trong tutorial này chúng ta sẽ bắt đầu từ đơn vị nhỏ nhất của pytorch là `Tensor`, sau đó xây dựng một mô hình neural net và thực tập với seq2seq

Để cài đặt trên máy, bạn có thể làm theo hướng dẫn [sau](https://pytorch.org/get-started/locally/). Trên colab đã có sẵn thư viện này

In [ ]:
import torch
import torch.nn as nn
import numpy as np

## 2. Tensor & các phép tính

`Tensor` là đơn vị nhỏ nhất trong `Pytorch`, là một ma trận nhiều chiều. Hầu hết các phép tính trên `Tensor` giống với `numpy`(shape, reshape, ones, zeros, indexing...) & broadcasting

In [ ]:
data = [[1, 2, 3], [4, 5, 6]]

tensor = torch.tensor(data)
print(f"{tensor}\nType: {tensor.dtype}\nShape:{tensor.shape}")

In [ ]:
tensor = torch.Tensor(3, 3)

Nhưng khác ở một số hàm

In [ ]:
tensor.dot(tensor)

Bên cạnh đó pytorch tensor còn có một số method khác

Các phép tính `inplace`

`device`

### 2.2 Autograd

Autograd được đi kèm trong pytorch (giống với các framework ML khác). Chúng ta chỉ cần khai báo chiều forward, pytorch sẽ tự biết cách tính gradients

In [ ]:
x = torch.tensor([6.], requires_grad=True)
print(x.grad)

In [ ]:
y = x ** 3 + 2

In [ ]:
y.backward()

In [ ]:
x.grad # dy/dx = 3*x^2 = 3 * 36 = 108

In [ ]:
z = x ** 2

In [ ]:
z.backward()

In [ ]:
x.grad

In [ ]:
x

Câu hỏi: Tại sao ?

In [ ]:
a = torch.tensor([5., 2.], requires_grad=True)

In [ ]:
b = a * 2 + 5
c = b * 3
d = c.sum()

In [ ]:
d.backward()

In [ ]:
d.grad_fn

In [ ]:
c.grad_fn

In [ ]:
c.data_ptr()

In [ ]:
d.grad_fn.next_functions

In [ ]:
d.grad_fn.next_functions[0][0].next_functions

## 3. Xây dựng neural networks

Là một framework deep learning, `Pytorch` đi kèm sẵn các class trong `torch.nn` để xây dựng một mạng DL phức tạp. Để thuận tiện chúng ta sẽ dùng `nn`

In [ ]:
import torch.nn as nn

### 3.1 Layers

In [ ]:
in_feature = 5
out_feature = 2

input = torch.ones(6, 3, in_feature)

linear = nn.Linear(in_feature, out_feature)

output = linear(input)

In [ ]:
output.shape

### 3.2 Các module khác

Bên cạnh Linear, pytorch còn đi kèm các layer khác. Các layer phổ biến là

`nn.Conv1d`,
`nn.Conv2d`,
`nn.MaxPool2d`,
`nn.Embedding`,
`nn.LSTM`,
`nn.LSTMCell`,
`nn.MultiheadAttention`
...
và các activation `nn.Sigmoid`, `nn.Softmax`,...

In [ ]:
output

In [ ]:
output.sigmoid()

In [ ]:
nn.Sigmoid()(output)

In [ ]:
my_model  =nn.Sequential(
    nn.Linear(in_feature, out_feature),
    nn.Sigmoid()
)

In [ ]:
my_model(input)

### 3.3 Custom module

Chúng ta có thể tự xây dựng các layers khác bằng cách extend từ `nn.Module`. Các bước bạn cần làm để tạp 1 custom module là:

1. Khai báo các weight, layer cần dùng trong constructor
2. Xây dựng lan truyền thuận trong hàm `forward`

In [ ]:
from typing import List
import torch.nn.functional as F

In [ ]:
class MyMLP(nn.Module):
    def __init__(self, input_size:int, 
                 hidden_sizes: List[int], 
                 output_size: int):
        # Call to the __init__ function of the super class
        super(MyMLP, self).__init__()
        layers = []
        current_dim = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_dim, hidden_size))
            layers.append(nn.ReLU())
            current_dim = hidden_size
        
        layers.append(nn.Linear(current_dim, output_size))
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.layers(x)
        out = F.softmax(x, dim=-1)
        return out

In [ ]:
input_size = 32
hidden_sizes = [64, 64, 32]
output_size = 5
batch_size = 16

In [ ]:
inputs = torch.rand((batch_size, input_size))

In [ ]:
model = MyMLP(input_size, hidden_sizes, output_size)

In [ ]:
print(model)

Bài tập 1: Sửa lỗi inference của model sau:

In [ ]:
class MyMLPWide(nn.Module):
    def __init__(self, input_size:int, 
                 hidden_sizes: List[int], 
                 output_size: int):
        # Call to the __init__ function of the super class
        super(MyMLPWide, self).__init__()
        wide_layers = []
        current_dim = input_size
        for hidden_size in hidden_sizes:
            wide_layers.append(nn.Linear(input_size, hidden_size))
        
        self.output_layer = nn.Linear(np.sum(hidden_sizes), output_size)
        self.wide_layers = nn.ModuleList(wide_layers)
        
    def forward(self, x):
        outputs = []
        for layer in self.wide_layers:
            outputs.append(layer(x))
        
        outputs = torch.cat(outputs)
        out = self.output_layer(outputs)
        return out

In [ ]:
model = MyMLPWide(input_size, hidden_sizes, output_size)

In [ ]:
print(model)

In [ ]:
model(inputs)

In [ ]:
# model(inputs) Có shape bao nhiêu ? 

### 3.4 Optimizer & Loss function

In [ ]:
input_size = 32
hidden_sizes = [64, 64, 32]
output_size = 1
batch_size = 16

In [ ]:
model = MyMLPWide(input_size, hidden_sizes, output_size)

In [ ]:
import torch.optim as optim

In [ ]:
inputs = torch.rand(batch_size, input_size)
target = torch.ones(batch_size, 1)

optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
n_epochs = 10
for epoch in range(n_epochs):
    optimizer.zero_grad()
    y_pred = model(inputs)
    loss = loss_fn(y_pred, target)
    print(loss)
    loss.backward()
    optimizer.step()

## 4. Demo Seq2Seq & Variants

https://colab.research.google.com/drive/1qzyPAt7YUVULEsOCIFUJ6DHgC58pgBYh?usp=sharing

In [ ]:
### 4.2 Encoder

In [ ]:
### 4.3 Decoder

In [ ]:
### 4.4 Seq2Seq & Train

In [ ]:
### 4.5 Inference

In [ ]:
## 5. Practice 1

In [ ]:
## 6. Practice 2

## 5. Tài liệu đọc thêm

1. Autograd from scratch: [Grokking deeplearning - Chapter 13](https://livebook.manning.com/book/grokking-deep-learning/chapter-13?origin=product-toc)

2. Official pytorch tutorial: [tutorials](https://pytorch.org/tutorials/)

3. Deep learning with pytorch [book pdf](https://www.google.com/search?client=firefox-b-d&q=deep+learning+with+pytorch+book)